In [54]:

import numpy as np
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from enum import Enum, auto
from random import uniform, random
from math import cos, sin, pi, inf, sqrt
import os

#               粒子半径.
radius = 0.075
#              時間刻み.
dt = 0.01
#          正方形 Box の一辺の長さの半分.half = 5
half = 1
#            速度の大きさ.
speed = 1
#                       回復にかかる時間（回復率 γ の逆数）.time_to_recover = 2
time_to_recover = 0.05
#                   バッファ.buff = 300
buff = 1


#                                            図.
fig = plt.figure(figsize=plt.figaspect(1/1))
#                                            図の背景色は黒.
fig.set_facecolor('black')


class State(Enum):
    Susceptible = auto()
    Infected = auto()
    Recovered = auto()


class Person:
#    vars(spersons[0])要素,len=７
#    co: np.ndarray        現在の 2 次元座標.
#    coords: np.ndarray        過去の座標を全て記憶.
#    arg: float        初期速度の偏角.
#    standstill: bool        停止しているか否か.
#    vel: np.ndarray        現在の速度（2 次元）.
#    state: State        現在の感染状態.
#    infection_start: int        感染が始まった時間の index

    def __init__(
        self,
        co,
        state,
        standstill
    ):
        
        self.co = co
        self.coords = np.array([self.co])
        self.arg = uniform(-pi, pi)
        self.standstill = standstill
        if self.standstill:
            # 停止させるのであれば初期速度は (0.0, 0.0).
            vel = np.array([0.0, 0.0])
        else:
            # そうでないのであれば偏角（self.arg）方向に大きさ speed で等速運動.
            vel = np.array([speed * cos(self.arg), speed * sin(self.arg)])
        self.vel = vel
        self.state = state
        self.infection_start = inf

    def move(
        self
    ):
        """
        もし self が停止しているのならば, 動かす.
        """
        if self.standstill:
            self.vel = np.array([speed * cos(self.arg), speed * sin(self.arg)])


class Population:

    def __init__(
        self,
        number,
        comeback,
        p=None
    ):
        
        self.number = number
        self.R0 = (radius * (sqrt(2) * speed) * time_to_recover * self.number) / (half ** 2)
        print("R0: %f" % self.R0)
        self.persons = []
        self.p = p
        init_coords = []
        if self.p is not None:
            self.Re = (1.0 - self.p) * (1.0 + (sqrt(2) - 1) * self.p) * self.R0
            print("Re: %f" % self.Re)
        # 総人口数分の人間を生成.__for[0]in[0,1,2,3,],menber[4]
        for i in range(self.number):
            if i == 0:
                # 最初の人間は感染者として初期条件を設定.
                # 初期座標は原点に設定.
                init_co = np.array([0.0, 0.0])
                infected = Person(co=init_co,
                                  state=State.Infected,
                                  standstill=False)
                infected.infection_start = 0
                self.persons.append(infected)
            else:
                # それ以外の人間は未感染者として初期条件を設定.
                whi=0
                while True:
                    # 以前の人間の初期座標とは異なるまで while ループで座標をランダムに生成.
                    init_co = np.array([uniform(-half, half), uniform(-half, half)])
                    appropriate = True
                    print("115_whi, for-in =len(init_coords) ",whi,len(init_coords))
                    foi=0
                    for pre_init_co in init_coords:
                        dist = np.linalg.norm(init_co - pre_init_co, ord=2)
                        # 3 *（半径）の時は不適合 # 最低 2 *（半径）で重なる.
                        # 不適距離を大きすぎる、かつ, 人口数が大きい、とき
                        # for-in中に、新規座標uniform不適合⇒while,uniformからやり直し
                        if dist <= 3 * radius:
                            appropriate = False
                            print("    124 _for-in =",foi," appropriate = ",appropriate)
                        print("    125 _for-in =",foi," appropriate = ",appropriate)
                        foi += 1
                    if appropriate:
                        print("128 _while_break_No =",whi,"\n")
                        break
                    
                    print("131 _whileNo, appropriate=",whi," , ",appropriate,"\n")
                    whi += 1
                    
                # 行動制限の割合（p）を指定していない時は全ての人間を動かす.
                # そうでない場合は, 一様ランダムに 0.0 ~ 1.0 の数を生成し,
                # その値が行動制限の割合以下ならば停止させ, そうでない時は動かす.
                if self.p is None:
                    standstill = False
                elif random() < self.p:
                    standstill = True
                else:
                    standstill = False
                self.persons.append(Person(co=init_co,
                                           state=State.Susceptible,
                                           standstill=standstill))
                print("146_menber-i ,while, len(persons(vars7)= ",i,",",whi,",",len(self.persons))
                
            init_coords.append(init_co)
            print("\n  149_menber-i,init_coords= ",i,",",init_coords,"\n")
            
        self.times = np.array([0])
        self.zeros = np.array([0])
        self.I = np.array([1])
        self.S = np.array([self.number - 1])
        self.R = np.array([0])
        self.comeback = comeback
        
        print("158_menber-i,len(self.persons)= ",i,",",len(self.persons))
        print("159_menber-i,vars(self.persons[i])= ",i,",",vars(self.persons[i]))
        print("160_menber-i,len(self.persons[i])= ",i,",",len(vars(self.persons[i])),"\n")

    def simulate_sir(
        self,
        threshold=2
    ):
        
        # 時間の index.
        i = 0
        # 感染者数が閾値（threshold）を超えたか否か.
        exceeded_threshold = False
        # 行動制限を解除したか否か.
        comebacked = False
        # 感染が収束した時の時間のインデックス.
        subsided = None
        # 感染が収束してからバッファの分だけ時間のインデックスが進むまで SIR モデルのシミュレーションを行う.
        while True:
            # 各人間に対し, 感染が始まってから回復時間経った時には,
            # その人間の感染状態を免疫獲得状態に変更する.
            # そして各人間を移動させる.
            # 正方形 Box の壁に当たった場合には反射させる.
            for person in self.persons:
                if (i - person.infection_start) * dt > time_to_recover:
                    person.state = State.Recovered
                # 速度の時間刻み dt のスカラー倍分進む.
                person.co += dt * person.vel
                # x 軸方向での反射.
                if abs(person.co[0]) > half:
                    person.co[0] = np.sign(person.co[0]) * half
                    person.vel[0] *= -1
                # y 軸方向での反射.
                if abs(person.co[1]) > half:
                    person.co[1] = np.sign(person.co[1]) * half
                    person.vel[1] *= -1
            # 各人間の接触を判定する.
            # 人間たちの index を全て集めたものを indices とし,
            # indices の先頭の index を取り出し k とする.
            # その k を index として持つ人間を person とし,
            # 残った indices の各 index を l として,
            # その l を index として持つ人間を other とする.
            # person と other が接触している場合, その 2 人を反射させるが,
            # もし一方が未感染状態で, もう一方が感染状態のときには,
            # 未感染状態の方を感染状態にする.
            indices = list(range(self.number))
            while len(indices) > 0:
                k = indices.pop(0)
                person = self.persons[k]
                for l in indices:
                    other = self.persons[l]
                    vec = person.co - other.co
                    dist = np.linalg.norm(vec, ord=2)
                    # person と other の距離が 2 *（半径）より小さいとき接触したと判定.
                    # 簡単のため 3 人以上の同時接触は考えない.
                    if 0 < dist < 2 * radius:
                        normal = vec / dist
                        # person と other が重ならないようにするための補正（correction）.
                        correction = (radius - (dist / 2)) * normal
                        person.co += correction
                        other.co -= correction
                        # 物理的な完全弾性衝突の場合は -= np.dot(person.vel - other.vel, normal) * normal.
                        person.vel -= (2 * np.dot(person.vel, normal)) * normal
                        # 物理的な完全弾性衝突の場合は -= np.dot(other.vel - person.vel, normal) * normal.
                        other.vel -= (2 * np.dot(other.vel, normal)) * normal
                        indices.remove(l)
                        # 感染状態を必要であれば変化させる.
                        if person.state == State.Infected and other.state == State.Susceptible:
                            other.state = State.Infected
                            other.infection_start = i
                        elif person.state == State.Susceptible and other.state == State.Infected:
                            person.state = State.Infected
                            person.infection_start = i
                        break
            # 時間の index を一つ進める.
            i += 1
            # 総感染者数.
            I = 0
            # 総未感染者数.
            S = 0
            # 総免疫獲得者数.
            R = 0
            for person in self.persons:
                # 各人間に対し, 現在の座標を追加記憶させる.
                person.coords = np.vstack([person.coords, person.co])
                if person.state == State.Infected:
                    I += 1
                elif person.state == State.Susceptible:
                    S += 1
                elif person.state == State.Recovered:
                    R += 1
            # もし総感染者数 I が感染者数の閾値（threshold）を
            # 初めて超えた場合, exceeded_threshold を真とする.
            if threshold < I and not exceeded_threshold:
                exceeded_threshold = True
            # 行動制限を解除する場合（self.comeback が真の場合）,
            # もしまだ行動制限を解除しておらず（comebacked が偽）,
            # 総感染者数 I が感染者数の閾値（threshold）を既に超えており（exceeded_threshold が真）
            # その後総感染者数 I が感染者数の閾値（threshold）を下回った場合,
            # 停止させている人間は全て動かす.
            if I <= threshold and self.comeback and not comebacked and exceeded_threshold:
                for person in self.persons:
                    person.move()
                comebacked = True
            # 総感染者数が 0 つまり感染が収束した場合,
            # 感染収束時の時間の index を subsided に記憶させる.
            if I == 0 and subsided is None:
                subsided = i
            if comebacked:
                print("Simulating SIR %d (I: %d, S: %d, R: %d) (Comebacked)" % (i, I, S, R))
            else:
                print("Simulating SIR %d (I: %d, S: %d, R: %d)" % (i, I, S, R))
            # それぞれの数値計算結果を記憶.
            self.times = np.append(self.times, i * dt)
            self.zeros = np.append(self.zeros, 0)
            self.I = np.append(self.I, I)
            self.S = np.append(self.S, S)
            self.R = np.append(self.R, R)
            # 感染収束時の時間の index が subsided に記憶されており,
            # 時間の index（= i）が subsided とバッファの和になったとき,
            # シミュレーションを終了する.
            if subsided is not None and i == subsided + buff:
                self.end = i
                break


if __name__ == "__main__":
    population = Population(number=4,
                            comeback=True,
                            p=0.2)
    population.simulate_sir()
    # population.animate()
    
    

R0: 0.021213
Re: 0.018376

  149_menber-i,init_coords=  0 , [array([0., 0.])] 

115_whi, for-in =len(init_coords)  0 1
    125 _for-in = 0  appropriate =  True
128 _while_break_No = 0 

146_menber-i ,while, len(persons(vars7)=  1 , 0 , 2

  149_menber-i,init_coords=  1 , [array([0., 0.]), array([-0.50182998, -0.75404148])] 

115_whi, for-in =len(init_coords)  0 2
    125 _for-in = 0  appropriate =  True
    125 _for-in = 1  appropriate =  True
128 _while_break_No = 0 

146_menber-i ,while, len(persons(vars7)=  2 , 0 , 3

  149_menber-i,init_coords=  2 , [array([0., 0.]), array([-0.50182998, -0.75404148]), array([-0.87886902, -0.29447099])] 

115_whi, for-in =len(init_coords)  0 3
    125 _for-in = 0  appropriate =  True
    125 _for-in = 1  appropriate =  True
    124 _for-in = 2  appropriate =  False
    125 _for-in = 2  appropriate =  False
131 _whileNo, appropriate= 0  ,  False 

115_whi, for-in =len(init_coords)  1 3
    125 _for-in = 0  appropriate =  True
    125 _for-in = 1  app

<Figure size 288x288 with 0 Axes>